In [1]:
%pip install -q datasets langchain langchain-core langchain-community langchain-aws GitPython langchain-text-splitters langchain-chroma


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# load the swe-bench lite dataset
from datasets import load_dataset

dataset = load_dataset("princeton-nlp/SWE-bench_Lite")
example_row = dataset["test"][0]
list(example_row.keys())

['repo',
 'instance_id',
 'base_commit',
 'patch',
 'test_patch',
 'problem_statement',
 'hints_text',
 'created_at',
 'version',
 'FAIL_TO_PASS',
 'PASS_TO_PASS',
 'environment_setup_commit']

In [3]:
rows = dataset["test"].skip(0)
print(rows[0]["patch"])

diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py
--- a/astropy/modeling/separable.py
+++ b/astropy/modeling/separable.py
@@ -242,7 +242,7 @@ def _cstack(left, right):
         cright = _coord_matrix(right, 'right', noutp)
     else:
         cright = np.zeros((noutp, right.shape[1]))
-        cright[-right.shape[0]:, -right.shape[1]:] = 1
+        cright[-right.shape[0]:, -right.shape[1]:] = right
 
     return np.hstack([cleft, cright])
 


In [4]:
pred = {"model_name_or_path": "issue-review-local", "instance_id": "matplotlib__matplotlib-23314", "model_patch": "diff --git a/lib/mpl_toolkits/mplot3d/axes3d.py b/lib/mpl_toolkits/mplot3d/axes3d.py\n--- a/lib/mpl_toolkits/mplot3d/axes3d.py\n+++ b/lib/mpl_toolkits/mplot3d/axes3d.py\n@@ -2128,6 +2128,14 @@ class Axes3D(Axes):\n         return super().get_tightbbox(renderer, call_axes_locator=call_axes_locator,\n                                    bbox_extra_artists=bbox_extra_artists)\n \n+    def set_visible(self, visible):\n+        \"\"\"\n+        Set the axis visibility.\n+        \"\"\"\n+        super().set_visible(visible)\n+        # Update visibility of all 3D artists\n+        for artist in self._get_axis_list():\n+            artist.set_visible(visible)"}

print(pred["model_patch"])

diff --git a/lib/mpl_toolkits/mplot3d/axes3d.py b/lib/mpl_toolkits/mplot3d/axes3d.py
--- a/lib/mpl_toolkits/mplot3d/axes3d.py
+++ b/lib/mpl_toolkits/mplot3d/axes3d.py
@@ -2128,6 +2128,14 @@ class Axes3D(Axes):
         return super().get_tightbbox(renderer, call_axes_locator=call_axes_locator,
                                    bbox_extra_artists=bbox_extra_artists)
 
+    def set_visible(self, visible):
+        """
+        Set the axis visibility.
+        """
+        super().set_visible(visible)
+        # Update visibility of all 3D artists
+        for artist in self._get_axis_list():
+            artist.set_visible(visible)


In [5]:
# repo -> documents
import os

FILE_EXTENSIONS = [".py"]

# configure root folder for each repo
repo_to_top_folder = {
    "django/django": "django",
    "sphinx-doc/sphinx": "sphinx",
    "scikit-learn/scikit-learn": "scikit-learn",
    "sympy/sympy": "sympy",
    "pytest-dev/pytest": "pytest",
    "matplotlib/matplotlib": "matplotlib",
    "astropy/astropy": "astropy",
    "pydata/xarray": "xarray",
    "mwaskom/seaborn": "seaborn",
    "psf/requests": "requests",
    "pylint-dev/pylint": "pylint",
    "pallets/flask": "flask",
}

repo_path = f".playground/{example_row['repo'].split('/')[-1]}"


In [6]:
from git import Repo
from git.exc import GitCommandError
from pathlib import Path
from typing import List, Optional
from langchain_core.documents import Document
import hashlib


def hash_text(text: str):
    return hashlib.md5(text.encode()).hexdigest()


def clone_and_get_contents(
    clone_url: str, 
    repo_path: str, 
    instance_id: str,
    target_folder: Optional[str] = None,
    checkout: Optional[str] = None,
    file_extensions: Optional[List[str]] = None,
) -> List[Document]:
    """
    Clone a repository, checkout a specific commit, and return contents of files from a target folder.
    
    Args:
        repo_url: URL of the git repository
        commit_hash: Commit hash to checkout
        target_folder: Folder within the repository to traverse
        
    Returns:
        Dictionary mapping file paths to their contents
    """
    target_folder = target_folder or ""
    documents = []

    try:
        if os.path.isdir(os.path.join(repo_path, ".git")):
            repo = Repo(repo_path)
            # If the existing repository is not the same as the one we're trying to
            # clone, raise an error.
            if repo.remotes.origin.url != clone_url:
                raise ValueError(
                    "A different repository is already cloned at this path."
                )
        else:
            repo = Repo.clone_from(clone_url, repo_path)
        if checkout:
            repo.git.checkout(checkout)
        target_path = os.path.join(repo_path, target_folder)

        # ensure that the target folder exists
        if not os.path.exists(target_path):
            raise Exception(f"Target folder '{target_folder}' not found in repository")

        # Collect all file contents
        for root, _, files in os.walk(target_path):
            for file in files:
                # Get path relative to target folder
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, target_path)
                extension = Path(rel_path).suffix
                if file_extensions:
                    if extension not in file_extensions:
                        continue
                try:
                    # only get the text files
                    with open(full_path, "r", encoding="utf-8") as f:
                        page_content = f.read()
                        documents.append(
                            Document(
                                page_content=page_content, 
                                metadata={
                                    "file_path": os.path.join(target_folder, rel_path), 
                                    "file_type": extension,
                                    "instance_ids": [instance_id],
                                }
                            )
                        )
                except Exception as e:
                    # non-text files are skipped
                    continue

        return documents

    except GitCommandError as e:
        print(f"Git error occurred: {e}")
        return []
    except Exception as e:
        print(f"Error occurred: {e}")
        return []
    
docs = clone_and_get_contents(
    clone_url=f"https://github.com/{example_row['repo']}", 
    repo_path=repo_path, 
    instance_id=example_row["instance_id"],
    checkout=example_row["base_commit"], 
    target_folder=repo_to_top_folder[example_row["repo"]],
    file_extensions=FILE_EXTENSIONS,
)
len(docs)

889

In [7]:
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)

extension_to_language = {
    ".py": "python",
    ".java": "java", # etc
}

# split docs by language
docs_by_language = {}
for doc in docs:
    doc_language = extension_to_language[doc.metadata["file_type"]]
    if doc_language not in docs_by_language:
        docs_by_language[doc_language] = [doc]
    else:
        docs_by_language[doc_language].append(doc)

# splitter
split_docs = []
for language in docs_by_language:
    language_splitter = RecursiveCharacterTextSplitter.from_language(
        language=language, 
        chunk_size=8192, 
        chunk_overlap=256,
    )
    new_docs = language_splitter.split_documents(docs_by_language[language])
    for doc in new_docs:
        # use the file hash as the id
        doc.id = hash_text(doc.page_content)
    split_docs += new_docs
    
len(split_docs)

2441

In [8]:
from langchain_aws import ChatBedrockConverse, BedrockEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import InMemoryVectorStore, FAISS, Chroma
from langchain_chroma import Chroma
from langchain_community.docstore import InMemoryDocstore
from botocore.config import Config
import faiss
import json
import boto3

bedrock_config = Config(
    retries={
        "max_attempts": 100
    }
)

credentials_profile = os.getenv("AWS_PROFILE", "ediewald+1111-Admin")  # change for your AWS_PROFILE

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    credentials_profile_name=credentials_profile,
    model_kwargs={"dimensions": 256},
    config=bedrock_config,
)

llm_light = ChatBedrockConverse(
    model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    credentials_profile_name=credentials_profile,
    config=bedrock_config,
)

llm_heavy = ChatBedrockConverse(
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    credentials_profile_name=credentials_profile,
    config=bedrock_config,
)

# vectorstores per repo so that filters are most efficient
vectorstore_base_path = ".vectorstores/"
index_name = example_row["repo"].split("/")[-1]
vectorstore_path = os.path.join(vectorstore_base_path, index_name)
# vectorstore = Chroma(
#     collection_name=example_row["repo"].replace("/", "-"),
#     embedding_function=embeddings,
#     persist_directory=vectorstore_path,
# )

In [9]:
if os.path.exists(vectorstore_path):
    vectorstore = FAISS.load_local(vectorstore_path, embeddings=embeddings, index_name=index_name, allow_dangerous_deserialization=True)
else:
    index = faiss.IndexFlatL2(len(embeddings.embed_query("foo")))
    vectorstore = FAISS(
        embedding_function=embeddings,
        index=index,
        index_to_docstore_id={},
        docstore=InMemoryDocstore(),
    )

In [10]:
from typing import Literal
import os

os.environ["AWS_PROFILE"] = credentials_profile

def embed_cohere(texts: List[str], embed_type: Literal["search_document", "search_query"]) -> List[List[float]]:

    bedrock = boto3.client("bedrock-runtime")
    # cohere can take 96 texts per request
    chunks = [texts[i:i + 96] for i in range(0, len(texts), 96)]
    vectors = []

    for chunk in chunks:
        request_body = {
            "modelId": "cohere.embed-multilingual-v3",
            "contentType": "application/json",
            "accept": "application/json",
            "body": json.dumps({"texts": chunk, "embed_type": embed_type}),
        }

        vectors += bedrock.invoke_model(**request_body)["body"].read()["embeddings"]
    return vectors

vecs = embed_cohere([doc.page_content for doc in split_docs], "search_document")

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [input_type] not found#: extraneous key [embed_type] is not permitted#/texts/0: expected maxLength: 2048, actual: 5074#/texts/2: expected maxLength: 2048, actual: 7711#/texts/3: expected maxLength: 2048, actual: 4234#/texts/4: expected maxLength: 2048, actual: 8168#/texts/5: expected maxLength: 2048, actual: 5527#/texts/8: expected maxLength: 2048, actual: 7914#/texts/10: expected maxLength: 2048, actual: 2440#/texts/11: expected maxLength: 2048, actual: 3940#/texts/12: expected maxLength: 2048, actual: 2194#/texts/13: expected maxLength: 2048, actual: 7961#/texts/14: expected maxLength: 2048, actual: 7204#/texts/15: expected maxLength: 2048, actual: 5960#/texts/16: expected maxLength: 2048, actual: 5715#/texts/17: expected maxLength: 2048, actual: 3747#/texts/18: expected maxLength: 2048, actual: 7665#/texts/19: expected maxLength: 2048, actual:

In [ ]:
# parallel embeddings
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Any
import numpy as np


def parallel_embed(texts: List[str], embeddings, max_workers: int = 4) -> List[Any]:
    """
    Parallelize the embedding of texts using a configurable number of workers.
    
    Args:
        texts (List[str]): List of texts to embed
        max_workers (int): Maximum number of parallel workers (default: 4)
        
    Returns:
        List[Any]: List of embeddings in the same order as input texts
    """
    # Dictionary to keep track of the original order
    results_dict = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and store futures with their indices
        future_to_idx = {
            executor.submit(embeddings._embedding_func, text): idx
            for idx, text in enumerate(texts)
        }

        # Process completed futures and store results
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                result = future.result()
                results_dict[idx] = result
            except Exception as e:
                print(f"Error processing text at index {idx}: {str(e)}")
                results_dict[idx] = None

    # Return results in original order
    return [results_dict[i] for i in range(len(texts))]


def add_docs_to_store(vectorstore: FAISS, embeddings: BedrockEmbeddings, documents: List[Document], max_workers: int = 100):
    # find the existing docs, if any
    existing_docs = [doc for doc in vectorstore.get_by_ids([d.id for d in documents])]
    print(f"found {len(existing_docs)} existing docs")
    
    # only need to embed the new/updated files
    new_docs = [doc for doc in documents if doc not in existing_docs]
    print(f"found {len(new_docs)} new docs")
    
    assert len(documents) - len(existing_docs) == len(new_docs), "Num. new docs + num updated docs != num docs"
    
    # first, update the existing docs if necessary
    for doc in existing_docs:
        existing_instance_ids = vectorstore.docstore._dict[doc.id].metadata["instance_ids"]
        vectorstore.docstore._dict[doc.id].metadata["instance_ids"] = list(set(existing_instance_ids + doc.metadata["instance_ids"]))
    
    # now, add the new docs
    vectors = parallel_embed([doc.page_content for doc in new_docs], embeddings, max_workers=max_workers)
    # add embeddings to FAISS
    vectorstore.index.add(np.array(vectors, dtype=np.float32))
    # add metadata / docs to docstore
    vectorstore.docstore.add({doc.id: doc for doc in new_docs})
    # link metadata with FAISS via index_to_docstore_id
    starting_len = len(vectorstore.index_to_docstore_id)
    index_to_id = {starting_len + j: id_ for j, id_ in enumerate([doc.id for doc in new_docs])}
    vectorstore.index_to_docstore_id.update(index_to_id)


add_docs_to_store(vectorstore, embeddings, documents=split_docs)
# vectorstore.save_local(vectorstore_path, index_name=index_name)

In [ ]:
vectorstore.similarity_search("filter", filter=lambda x: 'django__django-d' in x["instance_ids"])

In [ ]:
print(example_row["problem_statement"])

In [16]:
# fault localization
from typing import TypedDict, Annotated

# file we want: django/core/management/commands/sqlmigrate.py
# print(example_row["patch"])

class CodeSearch(TypedDict):
    """The search queries to submit to the code search engine"""
    queries: Annotated[List[str], "The search queries (can be natural language or exact search)"]

PROMPT_FAULT_LOCALIZATION_SEARCH = """You are a senior software engineer working on addressing the following issue:

---BEGIN ISSUE---
{problem_statement}
---END ISSUE---

Your first objective is to identify the file(s) that you will need to edit in order to address the issue.

You have access to a code search tool that you can use to search through the codebase. Submit 3-5 search queries that you want to use to find the right file(s).
"""

llm_search_queries = llm_light.with_structured_output(CodeSearch)

res = llm_search_queries.invoke(PROMPT_FAULT_LOCALIZATION_SEARCH.format(problem_statement=example_row["problem_statement"]))
res

{'queries': ['separability_matrix',
  'astropy.modeling.separable',
  'separability implementation']}

In [17]:
relevant_docs = []
for query in res["queries"]:
    relevant_docs += vectorstore.similarity_search_with_score(query, k=5)
sorted_docs = [doc for doc in sorted(relevant_docs, key=lambda x: x[1], reverse=True)]
unique_files, file_scores = [], []
for doc, score in sorted_docs:
    file_path = doc.metadata["file_path"]
    if file_path not in unique_files:
        unique_files.append(file_path)
        file_scores.append(score)

# able to identify the right file, but we should assume it will not always be the top result
print("\n".join([f"{round(score, 2)} | {fp}" for fp, score in zip(unique_files, file_scores)]))

AssertionError: 

In [14]:
from typing import TypedDict, Annotated

class SolutionPatch(TypedDict):
    """A git patch to address the provided issue."""
    patch: Annotated[str, "The git patch."]
    

PROMPT_GENERATE_PATCH = """You are a senior software engineer addressing the following issue:

---BEGIN ISSUE---
{problem_statement}
---END ISSUE---

We've identified that the fix will likely require edits to one or more of the following file(s) (note that line numbers have been added for reference):

---BEGIN CODE FILES---
{file_candidates}
---END CODE FILES---

Instructions:
1. Carefully read the problem statement and the provided code files.
2. Create a git patch that will address the issue. You may only need to edit one of the files that was provided. Your patch should NOT include an `index` header."""

def get_code_file(repo_path: str, file_path: str, line_numbers: bool = True) -> str:
    contents = f"File: {file_path}\n\n"
    with open(os.path.join(repo_path, file_path), "r") as f:
        code_lines = f.read()
    for idx, line in enumerate(code_lines.splitlines()):
        prefix = f"L{idx+1}: " if line_numbers else ""
        contents += f"{prefix}{line}\n"
    return contents


file_candidates = "\n\n".join([get_code_file(repo_path, fp) for fp in unique_files])

llm_patch = llm_heavy.with_structured_output(SolutionPatch)

model_patch = llm_patch.invoke(
    PROMPT_GENERATE_PATCH.format(
        problem_statement=example_row["problem_statement"], 
        file_candidates=file_candidates
    ),
)["patch"]

NameError: name 'unique_files' is not defined

In [11]:
print(file_candidates)

NameError: name 'file_candidates' is not defined

In [ ]:
print("Target Patch:")
print(example_row["patch"])

print("Model Patch:")
print(model_patch)

In [20]:
def get_directory_structure(repo_path: str) -> List[str]:
    """
    Get the directory structure of a repository.

    Args:
        repo_path (str): Path to the repository

    Returns:
        List[str]: List of file paths relative to the repository root
    """
    file_paths = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            # Skip hidden files
            if file.startswith("."):
                continue
            # Get the relative path
            relative_path = os.path.relpath(os.path.join(root, file), repo_path)
            file_paths.append(relative_path)
    return file_paths

repo_path = ".playground/django/django"
len(get_directory_structure(repo_path))

3503